In [1]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


In [3]:
!pip install langchain_openai langchain.tools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 38.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 63.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 3.6 MB/s eta 0:00:00


In [4]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [25]:
import random
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def weather(query: str) -> str:
    """Procura pelo clima de uma cidade online"""
    weather = ["ensolarado", "nublado", "chuvoso", "neve"]
    result = random.choice(weather)
    return result

@tool
def multiply(a: int, b: int) -> int:
    """Multiplica dois números"""
    return a * b

@tool
def write_file(filename: str, content: str) -> None:
  """Escreva um arquivo de texto"""
  with open(filename, "w") as f:
    f.write(content)

@tool
def count_letters(word:str, letter:str)-> int:
  """Conta a quantidade de ocorrências de uma letra em uma palavra"""
  count = 0
  for char in word:
    if char == letter:
      count += 1
  return count

tools = [weather, multiply,write_file,count_letters]
llm_with_tools = llm.bind_tools(tools)

In [27]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Você é um assistente mas não sabe do clima ou fazer cálculos matemáticos.",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [28]:
from langchain.agents.format_scratchpad.openai_tools import format_to_openai_tool_messages
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

agent = (
    {
        "input": lambda x: x["input"],
        "agent_scratchpad": lambda x: format_to_openai_tool_messages(
            x["intermediate_steps"]
        ),
    }
    | prompt
    | llm_with_tools
    | OpenAIToolsAgentOutputParser()
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [22]:
agent_executor.invoke({"input": "Qual o clima em Recife?"})



> Entering new AgentExecutor chain...

Invoking: `weather` with `{'query': 'Recife'}`


nubladoO clima em Recife está nublado.

> Finished chain.


{'input': 'Qual o clima em Recife?',
 'output': 'O clima em Recife está nublado.'}

## Exercícios

### Exercício 1
Adicione uma função nas ferramentas do agente para criar arquivos de texto. Em seguida, teste o agente.

In [18]:
agent_executor.invoke({"input": "Crie um arquivo de texto contendo informações sobre o clima em recife"})



> Entering new AgentExecutor chain...

Invoking: `weather` with `{'query': 'Recife'}`


chuvoso
Invoking: `write_file` with `{'filename': 'clima_recife.txt', 'content': 'Informações sobre o clima em Recife:\n\nO clima em Recife está chuvoso.'}`


NoneO arquivo de texto contendo informações sobre o clima em Recife foi criado com sucesso. Ele contém a seguinte informação:

```
Informações sobre o clima em Recife:

O clima em Recife está chuvoso.
```

> Finished chain.


{'input': 'Crie um arquivo de texto contendo informações sobre o clima em recife',
 'output': 'O arquivo de texto contendo informações sobre o clima em Recife foi criado com sucesso. Ele contém a seguinte informação:\n\n```\nInformações sobre o clima em Recife:\n\nO clima em Recife está chuvoso.\n```'}

### Exercício 2
Crie um agente que conte a quantidade de ocorrências de uma determinada letra em uma palavra. Em seguida, teste o agente.

In [29]:
agent_executor.invoke({"input": "quantas letras R existem em strawberry?"})



> Entering new AgentExecutor chain...

Invoking: `count_letters` with `{'word': 'strawberry', 'letter': 'r'}`


3Existem 3 letras "R" na palavra "strawberry".

> Finished chain.


{'input': 'quantas letras R existem em strawberry?',
 'output': 'Existem 3 letras "R" na palavra "strawberry".'}